In [1]:
from __future__ import print_function
from keras.datasets import mnist

Using TensorFlow backend.


In [2]:
from bqplot import pyplot as plt
from bqplot import ColorScale, OrdinalScale, OrdinalColorScale, CATEGORY10
from ipywidgets import HBox, VBox, Layout, FloatProgress, Label, IntSlider, Dropdown
from IPython.display import display
import numpy as np

In [3]:
batch_size = 128
num_classes = 10
epochs = 20

# input image dimensions
img_rows, img_cols = 28, 28

# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [4]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (60000, 28, 28)
60000 train samples
10000 test samples


In [5]:
train_test_dd = Dropdown(options=['Train', 'Test'])
sel_slid = IntSlider(min=0, max=x_train.shape[0]-1, description='Index')
fig_map = plt.figure()
heat_map = plt.heatmap(color=x_train[0][::-1], scales={'color': ColorScale(scheme='Greys')},
                      axes_options={'x': {'visible': False},
                                    'y': {'visible': False},
                                    'color': {'visible': False}})

def slid_changed(change):
    heat_map.color = x_train[sel_slid.value][::-1]
    
def dd_changed(change):
    sel_slid.value = 0
    if train_test_dd.value == 'Train':
        sel_slid.max = x_train.shape[0]
    else:
        sel_slid.max = x_test.shape[0]
    
    slid_changed(None)
    
sel_slid.observe(slid_changed, 'value')
train_test_dd.observe(dd_changed, 'value')

In [6]:
VBox([HBox([train_test_dd, sel_slid]), fig_map])

In [7]:
from sklearn.manifold import TSNE

In [8]:
tsne_x = x_train[:1000, :, :].reshape(1000, 784)

In [9]:
axes_opt = {'x': {'visible': False},'y': {'visible': False}, 'color': {'visible': False}}

In [10]:
tsn = TSNE()
low_x = tsn.fit_transform(tsne_x)

In [19]:
tt_fig = plt.figure()
hmap_sc = ColorScale(scheme='Greys')
hmap = plt.heatmap(x_train[0][::-1], scales={'color': hmap_sc}, axes_options=axes_opt)

In [20]:
fig = plt.figure(title='TSNE with Hover')
ind = 1000
ord_sc = OrdinalColorScale(colors=CATEGORY10, domain=list(range(10)))
scat = plt.scatter(low_x[:ind, 0], low_x[:ind, 1], color=y_train[:ind], tooltip=tt_fig, scales={'color':ord_sc},
                  stroke='Black')

In [21]:
def hovered(name, value):
    if scat.hovered_point is not None:
        hmap.color = x_train[scat.hovered_point, :, :][::-1]
    
scat.on_hover(hovered)

In [22]:
fig